# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [63]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json 
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [64]:
city_df = pd.read_csv("output_data/cities.csv")
city_df.head()

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Clouds,Wind,Country,Date
0,0,san patricio,28.0170,-97.5169,92.12,60,16,17.65,US,1624662003
1,1,katsuura,35.1333,140.3000,79.23,77,82,6.96,JP,1624662004
2,2,hobyo,5.3505,48.5268,77.27,84,97,25.46,SO,1624662004
3,3,linjiang,41.8069,126.9078,69.89,72,97,2.35,CN,1624662004
4,4,rikitea,-23.1203,-134.9692,74.26,74,11,3.53,PF,1624662005


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [65]:
gmaps.configure(g_key)

In [66]:
location = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"]

In [67]:
fig = gmaps.figure()

heat_map = gmaps.heatmap_layer(
    location, 
    weights=city_df["Humidity"],
    dissipating=False,
    max_intensity=200,
    point_radius = 3)
fig.add_layer(heat_map)
print(fig)


Figure(layout=FigureLayout(height='420px'))


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [68]:
perfect_city_df = city_df.loc[(city_df["Wind"]<= 10) & (city_df["Clouds"] <= 0) & 
                                (city_df["Max Temperature"]>= 70) & (city_df["Max Temperature"]<= 80)].dropna()

perfect_city_df

,Unnamed: 0,City,Latitude,Longitude,Max Temperature,Humidity,Clouds,Wind,Country,Date
130,130,abu samrah,35.3029,37.1841,76.30,52,0,6.58,SY,1624662059
179,179,carinhanha,-14.3047,-43.7650,73.40,33,0,4.36,BR,1624662077
274,274,sainte-maxime,43.3000,6.6333,73.44,74,0,3.83,FR,1624662113
496,496,alexandria,31.2156,29.9553,77.43,77,0,6.82,EG,1624662194
498,498,misratah,32.3754,15.0925,79.83,65,0,0.54,LY,1624662012
513,513,ofaqim,31.3141,34.6203,74.73,69,0,3.36,IL,1624662199


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [69]:
hotel_df = perfect_city_df[["City", "Country", "Latitude", "Longitude"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
130,abu samrah,SY,35.3029,37.1841,
179,carinhanha,BR,-14.3047,-43.7650,
274,sainte-maxime,FR,43.3000,6.6333,
496,alexandria,EG,31.2156,29.9553,
498,misratah,LY,32.3754,15.0925,
513,ofaqim,IL,31.3141,34.6203,


In [70]:
lat = hotel_df.groupby('City')['Latitude']
#lng = perfect_city_df.loc["Longitude"]
lat.head()

130    35.3029
179   -14.3047
274    43.3000
496    31.2156
498    32.3754
513    31.3141
Name: Latitude, dtype: float64

In [71]:
hotels = []

for city in range(len(hotel_df["City"])):

    lat = perfect_city_df.groupby('City')["Latitude"]
    lng = perfect_city_df.groupby('City')["Longitude"]

    #city_coordinate = f"{lat},{lng}"

    params = {
        "location": hotel_df, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Hotel Name"] = hotels
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
130,abu samrah,SY,35.3029,37.1841,Nearest hotel not found
179,carinhanha,BR,-14.3047,-43.7650,Nearest hotel not found
274,sainte-maxime,FR,43.3000,6.6333,Nearest hotel not found
496,alexandria,EG,31.2156,29.9553,Nearest hotel not found
498,misratah,LY,32.3754,15.0925,Nearest hotel not found
513,ofaqim,IL,31.3141,34.6203,Nearest hotel not found


In [72]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [73]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))